In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("IMDB Dataset.csv")

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [5]:
df['review'] = df['review'].str.lower()

In [6]:
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

In [7]:
df['review'] = df['review'].apply(remove_html_tags)

In [8]:
def remove_url(text):
    pattern= re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [9]:
df['review'] = df['review'].apply(remove_url)

In [10]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
exclude = string.punctuation

In [12]:
def remove_punc(text):
    for char in exclude :
        text = text.replace(char,'')
    return text

In [13]:
df['review'] = df['review'].apply(remove_punc)

In [14]:
from textblob import TextBlob

In [15]:
def text_corrector(text):
    textBlb = TextBlob(text)
    corrected_text = str(textBlb.correct())
    return corrected_text
    

In [16]:
# df['review'] = df['review'].apply(text_corrector)

In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

In [19]:
X = df['review']
y = df['sentiment']

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [21]:
X_train.shape

(35000,)

In [22]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_padded = pad_sequences(tokenizer.texts_to_sequences(X_train),maxlen=200)
X_test_padded = pad_sequences(tokenizer.texts_to_sequences(X_test),maxlen=200)

In [23]:
len(tokenizer.word_index)

177591

In [24]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1


In [25]:
X_train_padded.shape

(35000, 200)

In [26]:
model = Sequential()
model.add(Embedding(input_dim=177592,output_dim=256, input_length=200))
model.add(LSTM(256, dropout=0.3, return_sequences=True))
model.add(LSTM(128, dropout=0.3))
model.add(Dense(1,activation='sigmoid'))

In [27]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [28]:
model.fit(X_train_padded, y_train, epochs=30, batch_size=64, validation_split=0.2)

Epoch 1/30
438/438 [==============================] - 68s 136ms/step - loss: 0.4783 - accuracy: 0.7760 - val_loss: 0.3297 - val_accuracy: 0.8633
Epoch 2/30
438/438 [==============================] - 59s 134ms/step - loss: 0.2053 - accuracy: 0.9224 - val_loss: 0.2954 - val_accuracy: 0.8771
Epoch 3/30
438/438 [==============================] - 58s 133ms/step - loss: 0.0783 - accuracy: 0.9746 - val_loss: 0.5521 - val_accuracy: 0.8661
Epoch 4/30
438/438 [==============================] - 58s 133ms/step - loss: 0.0547 - accuracy: 0.9827 - val_loss: 0.4954 - val_accuracy: 0.8533
Epoch 5/30
438/438 [==============================] - 59s 134ms/step - loss: 0.0584 - accuracy: 0.9798 - val_loss: 0.5232 - val_accuracy: 0.8576
Epoch 6/30
438/438 [==============================] - 59s 136ms/step - loss: 0.0449 - accuracy: 0.9852 - val_loss: 0.6557 - val_accuracy: 0.8451
Epoch 7/30
438/438 [==============================] - 59s 135ms/step - loss: 0.0144 - accuracy: 0.9954 - val_loss: 0.7778 - val_ac

In [29]:
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

469/469 [==============================] - 17s 36ms/step - loss: 1.1949 - accuracy: 0.8583
Test Loss: 1.1949410438537598
Test Accuracy: 0.8583333492279053
